## Create PyTorch Dataset

In [1]:
import transformers
print(transformers.__version__)

4.29.1


In [2]:
import pandas as pd
from glob import  glob
import json
from tqdm.notebook import trange, tqdm
from sklearn.model_selection import StratifiedKFold
import torch
import numpy as np
import random
import os
from transformers import get_linear_schedule_with_warmup, get_cosine_schedule_with_warmup
from torch.cuda.amp import GradScaler, autocast

In [3]:
image_path = glob('./train/images/*')

In [4]:
label_path = glob('./train/annotations/*')

In [5]:
assert len(image_path) == 60578

In [6]:
assert len(label_path)== 60578

### Understanding `max_patches` argument

The paper introduces a new paradigm for processing the input image. It takes the image and create `n_patches` aspect-ratio preserving patches, and concatenates the remaining sequence with padding tokens to finally get `max_patches` patches. It appears that this argument is quite crucial for training and evaluation, as the model becomes very sensitive to this parameter.

For the sake of our example, we will fine-tune a model with `max_patches=1024`.

Note that most of the `-base` models have been fine-tuned with `max_patches=2048`, and `4096` for `-large` models.

In [7]:
from torch.utils.data import Dataset, DataLoader
from PIL import Image
import pandas as pd
from tqdm.notebook import tqdm
MAX_PATCHES = 2048

class ImageCaptioningDataset(Dataset):
    def __init__(self, df, processor):
        self.dataset = df
        self.processor = processor

    def __len__(self):
        return len(self.dataset)

    def __getitem__(self, idx):
        row = self.dataset.iloc[idx, :]
        image = Image.open(row.image_path)
        #display(image)
        encoding = self.processor(images=image,
                                  #prompt
                                  #文本检测 去检测x y轴的label x-axis<X,X,X,X,> Y-axis<y,y,y,y>
                                  # DBnet
                                  text="Generate underlying data table of the figure below:",
                                  font_path="arial.ttf",
                                  return_tensors="pt",
                                  add_special_tokens=True, max_patches=MAX_PATCHES)
        
        encoding = {k:v.squeeze() for k,v in encoding.items()}
        encoding["text"] = row.label 
        return encoding

## Load model and processor

In [8]:
from transformers import Pix2StructProcessor, Pix2StructForConditionalGeneration

processor = Pix2StructProcessor.from_pretrained("google/matcha-base")
model = Pix2StructForConditionalGeneration.from_pretrained("google/matcha-plotqa-v2")

Now that we have loaded the processor, let's load the dataset and the dataloader:

In [9]:
def collator(batch):
    new_batch = {"flattened_patches": [], "attention_mask": []}
    texts = [item["text"] for item in batch]
    # print(texts)
    text_inputs = processor.tokenizer(text=texts,
                                      padding="max_length",
                                      return_tensors="pt",
                                      add_special_tokens=True,
                                      max_length=512,
                                      truncation=True
                                      )

    new_batch["labels"] = text_inputs.input_ids

    for item in batch:
        new_batch["flattened_patches"].append(item["flattened_patches"])
        new_batch["attention_mask"].append(item["attention_mask"])

    new_batch["flattened_patches"] = torch.stack(new_batch["flattened_patches"])
    new_batch["attention_mask"] = torch.stack(new_batch["attention_mask"])

    return new_batch

In [10]:
df = pd.read_csv('train_with_fold.csv')
print(len(df))
train_df = df[df['fold'] != 0]
train_df.reset_index(drop=True, inplace=True)

60578


In [13]:
class CFG:
    scheduler = 'cosine'  # ['linear', 'cosine']
    batch_scheduler = True
    num_cycles = 0.5  # 1.5
    num_warmup_steps = 0.2
    max_input_length = 130
    epochs = 10  # 5
    encoder_lr = 10e-6
    decoder_lr = 10e-6
    min_lr = 0.5e-6
    eps = 1e-6
    betas = (0.9, 0.999)
    weight_decay = 0
    num_fold = 5
    batch_size = 2
    seed = 1006
    num_workers = 2
    device='cuda:1'
    print_freq = 100

In [14]:
train_dataset = ImageCaptioningDataset(train_df, processor)
train_dataloader = DataLoader(train_dataset, shuffle=True, batch_size=CFG.batch_size, collate_fn=collator, pin_memory=True,
                                  prefetch_factor=40, num_workers=2)

In [15]:
def seed_everything(seed=42):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
seed_everything(CFG.seed)

In [16]:
def get_scheduler(cfg, optimizer, num_train_steps):
    cfg.num_warmup_steps = cfg.num_warmup_steps * num_train_steps
    if cfg.scheduler == 'linear':
        scheduler = get_linear_schedule_with_warmup(
            optimizer, num_warmup_steps=cfg.num_warmup_steps, num_training_steps=num_train_steps
        )
    elif cfg.scheduler == 'cosine':
        scheduler = get_cosine_schedule_with_warmup(
            optimizer, num_warmup_steps=cfg.num_warmup_steps, num_training_steps=num_train_steps,
            num_cycles=cfg.num_cycles
        )
    return scheduler

num_train_steps = int(len(train_dataset) / CFG.batch_size * CFG.epochs)


## Train the model

Let's train the model! Run the simply the cell below for training the model. We have observed that finding the best hyper-parameters was quite challenging and required a lot of trials and errors, as the model can easily enter in "collapse-model" (always predicting the same output, no matter the input) if the HP are not chosen correctly. In this example, we found out that using `AdamW` optimizer with `lr=1e-5` seemed to be the best approach.

Let's also print the generation output of the model each 10 epochs!

Bear in mind that the model took some time to converge, for instance to get decent results we had to let the script run for ~1hour.

In [17]:
EPOCHS = 10

optimizer = torch.optim.AdamW(model.parameters(), lr=1e-5)
scheduler = get_scheduler(CFG, optimizer, num_train_steps)
device = "cuda:1" if torch.cuda.is_available() else "cpu"
print(device)
model.to(device)
scaler = torch.cuda.amp.GradScaler()
model.train()

cuda:1


Pix2StructForConditionalGeneration(
  (encoder): Pix2StructVisionModel(
    (embeddings): Pix2StructVisionEmbeddings(
      (patch_projection): Linear(in_features=768, out_features=768, bias=True)
      (row_embedder): Embedding(4096, 768)
      (column_embedder): Embedding(4096, 768)
      (dropout): Dropout(p=0.0, inplace=False)
    )
    (encoder): Pix2StructVisionEncoder(
      (layer): ModuleList(
        (0): Pix2StructVisionLayer(
          (attention): Pix2StructVisionAttention(
            (query): Linear(in_features=768, out_features=768, bias=False)
            (key): Linear(in_features=768, out_features=768, bias=False)
            (value): Linear(in_features=768, out_features=768, bias=False)
            (output): Linear(in_features=768, out_features=768, bias=False)
          )
          (mlp): Pix2StructVisionMlp(
            (wi_0): Linear(in_features=768, out_features=2048, bias=False)
            (wi_1): Linear(in_features=768, out_features=2048, bias=False)
         

In [20]:
loss_file = open("loss.txt","w")

for epoch in range(CFG.epochs):
    print("Epoch:", epoch)
    for idx, batch in tqdm(enumerate(train_dataloader), total=len(train_dataloader)):
        labels = batch.pop("labels").to(device)
        flattened_patches = batch.pop("flattened_patches").to(device)
        attention_mask = batch.pop("attention_mask").to(device)
        optimizer.zero_grad()
        with torch.cuda.amp.autocast():
            outputs = model(flattened_patches=flattened_patches,
                            attention_mask=attention_mask,
                            labels=labels)

        loss = outputs.loss

        scaler.scale(loss).backward()
        #grad_norm = torch.nn.utils.clip_grad_norm_(model.parameters(), 1000)
        # Unscales gradients and calls
        # or skips optimizer.step()
        scaler.step(optimizer)
        # Updates the scale for next iteration
        scaler.update()
        scheduler.step()
        if idx % 100 == 0:
            print("Loss:", loss.item(), f'lr : {scheduler.get_lr()[0]:.6f} ', sep=' ')
            loss_file.write(f"Epoch: {epoch}, Iteration: {idx}, Loss: {loss.item()}\n")
            loss_file.flush()
        

    if (epoch + 1) % 1 == 0:
        torch.save(model.state_dict(), f'./matcha_v1/matcha_{epoch}.bin')

loss_file.close()
        

Epoch: 0


  0%|          | 0/24231 [00:00<?, ?it/s]

Loss: 0.6629443764686584 lr : 0.000010 
Loss: 0.7359920144081116 lr : 0.000010 
Loss: 1.233712911605835 lr : 0.000010 
Loss: 1.0288405418395996 lr : 0.000010 
Loss: 0.858933687210083 lr : 0.000010 
Loss: 1.2682100534439087 lr : 0.000010 
Loss: 0.7170652747154236 lr : 0.000010 
Loss: 0.6052768230438232 lr : 0.000010 
Loss: 0.481381893157959 lr : 0.000010 
Loss: 0.5676138401031494 lr : 0.000010 
Loss: 1.1902427673339844 lr : 0.000010 
Loss: 0.6400188207626343 lr : 0.000010 
Loss: 0.745026171207428 lr : 0.000010 
Loss: 1.1922599077224731 lr : 0.000010 
Loss: 0.5645087361335754 lr : 0.000010 
Loss: 0.31349435448646545 lr : 0.000010 


IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



Loss: 1.0860344171524048 lr : 0.000010 
Loss: 0.7062138319015503 lr : 0.000010 
Loss: 0.8567994832992554 lr : 0.000010 
Loss: 0.8080344200134277 lr : 0.000010 
Loss: 0.82718825340271 lr : 0.000010 
Loss: 0.40620002150535583 lr : 0.000010 
Loss: 0.68768310546875 lr : 0.000010 
Loss: 0.650678277015686 lr : 0.000010 
Loss: 0.7405890822410583 lr : 0.000010 
Loss: 1.1742229461669922 lr : 0.000010 
Loss: 0.9242892265319824 lr : 0.000010 
Loss: 1.201245665550232 lr : 0.000010 
Loss: 1.696030855178833 lr : 0.000010 
Loss: 0.9377285242080688 lr : 0.000010 
Loss: 0.912734866142273 lr : 0.000010 
Loss: 0.6134945750236511 lr : 0.000010 
Loss: 0.7733114957809448 lr : 0.000010 
Loss: 1.234897255897522 lr : 0.000010 
Loss: 0.94827800989151 lr : 0.000010 
Loss: 1.061649203300476 lr : 0.000010 
Loss: 0.836963951587677 lr : 0.000010 
Loss: 1.1883182525634766 lr : 0.000010 
Loss: 0.46650922298431396 lr : 0.000010 
Loss: 1.118831753730774 lr : 0.000010 
Loss: 0.8363049030303955 lr : 0.000010 
Loss: 0.9104

...

Loss: 0.6509323120117188 lr : 0.000000 
Loss: 1.474658489227295 lr : 0.000000 
Loss: 0.8113728761672974 lr : 0.000000 
Loss: 0.43169093132019043 lr : 0.000000 
Loss: 0.944196343421936 lr : 0.000000 
Loss: 1.5682525634765625 lr : 0.000000 
Loss: 0.452965646982193 lr : 0.000000 
Loss: 1.1235264539718628 lr : 0.000000 
Loss: 0.5490254759788513 lr : 0.000000 
Loss: 0.18809176981449127 lr : 0.000000 
Loss: 0.20429714024066925 lr : 0.000000 
Loss: 0.7552458047866821 lr : 0.000000 
Loss: 0.6732435822486877 lr : 0.000000 
Loss: 0.3152370750904083 lr : 0.000000 
Loss: 0.4704183042049408 lr : 0.000000 
Loss: 0.9963559508323669 lr : 0.000000 
Loss: 1.170369029045105 lr : 0.000000 
Loss: 0.7466986775398254 lr : 0.000000 
Loss: 0.8500944375991821 lr : 0.000000 
Loss: 0.6573537588119507 lr : 0.000000 
Loss: 0.7682428359985352 lr : 0.000000 
Loss: 0.6529017686843872 lr : 0.000000 
Loss: 1.7234169244766235 lr : 0.000000 
Loss: 0.8686783909797668 lr : 0.000000 
Loss: 1.2084954977035522 lr : 0.000000 
L

Loss: 0.8514475226402283 lr : 0.000000 
Loss: 1.293064832687378 lr : 0.000000 
Loss: 1.0143225193023682 lr : 0.000000 
Loss: 1.0422883033752441 lr : 0.000000 
Loss: 1.2476650476455688 lr : 0.000000 
Loss: 0.24910324811935425 lr : 0.000000 
Loss: 0.7972142696380615 lr : 0.000000 
Loss: 1.1901086568832397 lr : 0.000000 
Loss: 0.42412322759628296 lr : 0.000000 
Loss: 1.1317559480667114 lr : 0.000000 
Loss: 0.6861850619316101 lr : 0.000000 
Loss: 0.39656612277030945 lr : 0.000000 
Loss: 0.8621700406074524 lr : 0.000000 
Loss: 0.7056178450584412 lr : 0.000000 
Loss: 0.9518454074859619 lr : 0.000000 
Loss: 0.9984250664710999 lr : 0.000000 
Loss: 0.5631944537162781 lr : 0.000000 
Loss: 0.777550458908081 lr : 0.000000 
Loss: 0.31235307455062866 lr : 0.000000 
Loss: 0.9698060750961304 lr : 0.000000 
Loss: 0.6368464231491089 lr : 0.000000 
Loss: 0.47314193844795227 lr : 0.000000 
Loss: 0.7375641465187073 lr : 0.000000 
Loss: 0.8368582725524902 lr : 0.000000 
Loss: 0.2975318729877472 lr : 0.00000

  0%|          | 0/24231 [00:00<?, ?it/s]

Loss: 0.822554886341095 lr : 0.000000 
Loss: 1.1946192979812622 lr : 0.000000 
Loss: 0.8359084725379944 lr : 0.000000 
Loss: 0.6560397148132324 lr : 0.000000 
Loss: 0.8776852488517761 lr : 0.000000 
Loss: 0.635638952255249 lr : 0.000000 
Loss: 0.6803064942359924 lr : 0.000000 
Loss: 0.6456636190414429 lr : 0.000000 
Loss: 0.792097806930542 lr : 0.000000 
Loss: 0.6495094895362854 lr : 0.000000 
Loss: 0.33406782150268555 lr : 0.000000 
Loss: 0.8484772443771362 lr : 0.000000 
Loss: 0.6731852293014526 lr : 0.000000 
Loss: 0.8136777877807617 lr : 0.000000 
Loss: 0.24912846088409424 lr : 0.000000 
Loss: 1.0719871520996094 lr : 0.000000 
Loss: 0.6304479837417603 lr : 0.000000 
Loss: 0.7944542169570923 lr : 0.000000 
Loss: 1.0570679903030396 lr : 0.000000 
Loss: 1.469242811203003 lr : 0.000000 
Loss: 0.15994170308113098 lr : 0.000000 
Loss: 0.8354843258857727 lr : 0.000000 
Loss: 1.0803004503250122 lr : 0.000000 
Loss: 0.3736814856529236 lr : 0.000000 
Loss: 0.4584267735481262 lr : 0.000000 
L

Loss: 0.26986783742904663 lr : 0.000001 
Loss: 0.7213385105133057 lr : 0.000001 
Loss: 1.2187060117721558 lr : 0.000001 
Loss: 0.7964249849319458 lr : 0.000001 
Loss: 0.6108760237693787 lr : 0.000001 
Loss: 0.7462961077690125 lr : 0.000001 
Loss: 0.7796894907951355 lr : 0.000001 
Loss: 1.0889841318130493 lr : 0.000001 
Loss: 0.2672808766365051 lr : 0.000001 
Loss: 0.35482853651046753 lr : 0.000001 
Loss: 0.9952284097671509 lr : 0.000001 
Loss: 1.1094893217086792 lr : 0.000001 
Loss: 0.2425563782453537 lr : 0.000001 
Loss: 1.1115297079086304 lr : 0.000001 
Loss: 1.2193368673324585 lr : 0.000001 
Loss: 1.014015555381775 lr : 0.000001 
Loss: 0.7908509373664856 lr : 0.000001 
Loss: 1.1475123167037964 lr : 0.000001 
Loss: 0.6541234254837036 lr : 0.000001 
Loss: 1.2704393863677979 lr : 0.000001 
Loss: 0.9871131777763367 lr : 0.000001 
Loss: 0.9672818183898926 lr : 0.000001 
Loss: 0.6557827591896057 lr : 0.000001 
Loss: 1.2765076160430908 lr : 0.000001 
Loss: 0.8192527890205383 lr : 0.000001 